# Easily create an AIConfig from existing Openai code
1. Basic usage
2. Load existing aiconfig and continue
3. Capture function calling
4. Use Client API
5. Save to existing AIConfig (no JSON)

## 1. Basic usage

In [ ]:
# Install AIConfig package
!pip3 install python-aiconfig

# Create ~/.env file with this line: export OPENAI_API_KEY=<your key here>
# You can get your key from https://platform.openai.com/api-keys 
import openai
import dotenv
import os
dotenv.load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from aiconfig.ChatCompletion import get_completion_create_wrapped_openai

output_path = "my-first-aiconfig.json"

# replace openai import with this
openai = get_completion_create_wrapped_openai(
    output_aiconfig_ref=output_path,
)

def run_my_existing_openai_app(user_message: str):
    completion_params = {
        "model": "gpt-3.5-turbo",
        "top_p": 1,
        "max_tokens": 3000,
        "temperature": 1,
        "stream": False,
        "messages": [
            {
                "content": user_message,
                "role": "user",
            }
        ],
    }

    response = openai.chat.completions.create(**completion_params) # Creates a config saved to default path `aiconfig.json`
    print("Chat Completion Response: ")
    print(type(response))

In [ ]:
# Run your code as usual
run_my_existing_openai_app("Tell me a joke about apples")

In [ ]:
import json

print("my-first-aiconfig.json now represents your existing app:\n")
result = json.load(open(output_path))
print(
    json.dumps(result, indent=2)
)

In [ ]:
import json

print("Zoom in on the prompt:\n")
print(
    json.dumps(result["prompts"][0], indent=2)
)

## 2. Continue from existing aiconfig file

In [ ]:
# Run your code as usual
run_my_existing_openai_app("Tell a joke about apples in Shakespearian English.")

In [ ]:
print("my-first-aiconfig.json has your existing prompts:\n")

prompts = json.load(open(output_path))["prompts"]
print(
    json.dumps(prompts[0], indent=2)
)

In [ ]:
print("And your new prompt:\n")
print(
    json.dumps(prompts[1], indent=2)
)

# 3. Capture function calling

### My existing app using function calling

In [ ]:
# Function Call Capture


from typing import Any

output_path = "my-function-calling-aiconfig.json"

# replace openai import with this
openai = get_completion_create_wrapped_openai(
    output_aiconfig_ref=output_path,
)


def get_current_weather(location: str, unit: str) -> dict[str, Any]:
    return { "temperature": 22, "unit": "celsius", "description": "Sunny" }


def run_my_existing_weather_function_calling_app():
    completion_params = {
        "model": "gpt-3.5-turbo-0613",
        "messages": [{"role": "user", "content": "What is the weather like in Boston?"}],
        "functions": [
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
    }

    response = openai.chat.completions.create(**completion_params) 

    function_call_response = get_current_weather(location="Boston", unit="celsius")
    print(response)

    completion_params = {
      "model": "gpt-3.5-turbo-0613",
      "messages": [
        {"role": "user", "content": "What is the weather like in Boston?"},
        {"role": "assistant", "content": 'null', "function_call": {
              "name": "get_current_weather",
              "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
            }},
        {"role": "function", "name": "get_current_weather", "content": str(function_call_response)}

      ],
      "functions": [
        {
          "name": "get_current_weather",
          "description": "Get the current weather in a given location",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA"
              },
              "unit": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"]
              }
            },
            "required": ["location"]
          }
        }
      ]
    }

    response = openai.chat.completions.create(**completion_params) 
    print(type(response))

In [ ]:
# Run your code as usual
run_my_existing_weather_function_calling_app()

In [ ]:
print("Inspect my-function-calling-aiconfig.json:\n")

prompts = json.load(open(output_path))["prompts"]
print(
    json.dumps(prompts[0], indent=2)
)

## 4. Use Client API

In [ ]:
from aiconfig.ChatCompletion import get_completion_create_wrapped_openai_client


output_path = "my-aiconfig-from-Client-API.json"

client = get_completion_create_wrapped_openai_client(output_path)

def run_my_existing_client_api_app():
    completion_params = {
                "model": "gpt-3.5-turbo",
                "top_p": 1,
                "max_tokens": 3000,
                "temperature": 1,
                "stream": False,
                "messages": [
                    {
                        "content": "Compare and contrast bananas and cucumbers.",
                        "role": "user",
                    }
                ],
            }
    response = client.chat.completions.create(**completion_params)
    print(type(response))

    


In [ ]:
# Run your code as usual
run_my_existing_client_api_app()

In [ ]:
import json

print(f"Inspect {output_path}:\n")

prompts = json.load(open(output_path))["prompts"]
print(
    json.dumps(prompts[0], indent=2)
)

## 5. Save to existing AIConfig (no JSON)

In [ ]:
from aiconfig.ChatCompletion import get_completion_create_wrapped_openai
from aiconfig.Config import AIConfigRuntime

existing_aiconfig = AIConfigRuntime.create()

# replace openai import with this
openai = get_completion_create_wrapped_openai(
    output_aiconfig_ref=existing_aiconfig,
)

def run_my_existing_openai_app(user_message: str):
    completion_params = {
        "model": "gpt-3.5-turbo",
        "top_p": 1,
        "max_tokens": 3000,
        "temperature": 1,
        "stream": False,
        "messages": [
            {
                "content": user_message,
                "role": "user",
            }
        ],
    }

    response = openai.chat.completions.create(**completion_params) # Creates a config saved to default path `aiconfig.json`
    print("Chat Completion Response: ")
    print(type(response))

In [ ]:
run_my_existing_openai_app("Are tomatoes fruits?")

In [ ]:
await existing_aiconfig.run("prompt_0")

print("Result:")
print(existing_aiconfig.get_output_text("prompt_0"))